# Health Registry Data Profiling and Cleaning

This notebook contains exploratory data analysis (EDA) and a reproducible cleaning pipeline for the National Health Facility Registry dataset.


In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)


## 1. Data Loading


In [ ]:
# Load the raw data
df_raw = pd.read_csv('health_registry.csv')

print(f"Dataset shape: {df_raw.shape}")
print(f"\nColumns: {list(df_raw.columns)}")
print(f"\nFirst few rows:")
df_raw.head()


## 2. Exploratory Data Analysis

### 2.1 Data Overview


In [ ]:
# Basic information
print("Dataset Info:")
print(f"Shape: {df_raw.shape[0]} rows × {df_raw.shape[1]} columns")
print(f"\nMemory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nData types:\n{df_raw.dtypes}")
print(f"\nNon-null counts:\n{df_raw.count()}")


### 2.2 Missingness Analysis


In [ ]:
# Count missing values (including empty strings and 'NULL' strings)
missing_analysis = pd.DataFrame({
    'Total_Rows': len(df_raw),
    'Null_Count': df_raw.isnull().sum(),
    'Empty_String_Count': (df_raw == '').sum(),
    'NULL_String_Count': (df_raw == 'NULL').sum(),
    'N/A_Count': (df_raw == 'N/A').sum()
})

missing_analysis['Total_Missing'] = (
    missing_analysis['Null_Count'] + 
    missing_analysis['Empty_String_Count'] + 
    missing_analysis['NULL_String_Count']
)
missing_analysis['Missing_Percentage'] = (
    missing_analysis['Total_Missing'] / missing_analysis['Total_Rows'] * 100
)

print("Missingness Analysis:")
print(missing_analysis.round(2))


In [ ]:
# Exact duplicates
exact_duplicates = df_raw.duplicated(keep=False)
exact_dup_count = exact_duplicates.sum()

print(f"Exact duplicate rows: {exact_dup_count}")
print(f"Unique duplicate groups: {df_raw[exact_duplicates].drop_duplicates().shape[0]}")

# Sample of exact duplicates
if exact_dup_count > 0:
    print("\nSample of exact duplicates:")
    df_raw[exact_duplicates].head(10)


### 2.4 Data Quality Issues by Column

Let's examine each column for quality issues:


In [ ]:
# Facility ID analysis
print("=== Facility ID Analysis ===")
print(f"Unique values: {df_raw['facility_id'].nunique()}")
print(f"\nSample values:\n{df_raw['facility_id'].value_counts().head(20)}")

# Check for patterns
print(f"\nNull/empty/NULL: {(df_raw['facility_id'].isna() | (df_raw['facility_id'] == '') | (df_raw['facility_id'] == 'NULL')).sum()}")
print(f"\nValues starting with '#': {df_raw['facility_id'].str.startswith('#', na=False).sum()}")
print(f"Values starting with 'HF-': {df_raw['facility_id'].str.startswith('HF-', na=False).sum()}")
print(f"Hex-like values (6 char alphanumeric): {df_raw['facility_id'].str.match(r'^[0-9a-f]{6}$', na=False).sum()}")


In [ ]:
# Facility Name analysis
print("=== Facility Name Analysis ===")
print(f"Unique values: {df_raw['facility_name'].nunique()}")

# Check for emojis and special characters
sample_names = df_raw['facility_name'].dropna().head(50)
print("\nSample facility names with issues:")
for name in sample_names:
    if any(ord(char) > 127 for char in str(name)) or '\n' in str(name):
        print(f"  - {repr(name)}")


In [ ]:
# Facility Type analysis
print("=== Facility Type Analysis ===")
print(f"Unique values: {df_raw['facility_type'].nunique()}")
print(f"\nValue counts:\n{df_raw['facility_type'].value_counts()}")


In [ ]:
# Capacity analysis
print("=== Capacity Analysis ===")
print(f"Unique values: {df_raw['capacity'].nunique()}")
print(f"\nSample values:\n{df_raw['capacity'].value_counts().head(30)}")


In [ ]:
# Region analysis
print("=== Region Analysis ===")
print(f"Unique values: {df_raw['region'].nunique()}")
print(f"\nValue counts:\n{df_raw['region'].value_counts().head(30)}")

# Check for reversed text patterns
reversed_pattern = df_raw['region'].str.contains(r'\.tS', na=False, case=False)
print(f"\nPotential reversed text (contains '.tS'): {reversed_pattern.sum()}")
if reversed_pattern.sum() > 0:
    print("\nSample reversed regions:")
    print(df_raw[reversed_pattern]['region'].unique()[:10])


In [ ]:
# Date analysis
print("=== Date Format Analysis ===")
print("Licence Issue Date sample values:")
print(df_raw['licence_issue_date'].value_counts().head(20))
print("\nInspection Date sample values:")
print(df_raw['inspection_date'].value_counts().head(20))


In [ ]:
# GPS Location analysis
print("=== GPS Location Analysis ===")
print(f"Unique formats detected:")

gps_sample = df_raw['gps_location'].dropna().head(100)
print("\nSample GPS formats:")
for gps in gps_sample.unique()[:20]:
    print(f"  - {repr(gps)}")

# Count format types
point_format = df_raw['gps_location'].str.contains('POINT', na=False).sum()
degree_format = df_raw['gps_location'].str.contains('°', na=False).sum()
comma_format = df_raw['gps_location'].str.contains(',', na=False).sum() - point_format
semicolon_format = df_raw['gps_location'].str.contains(';', na=False).sum()

print(f"\nFormat counts:")
print(f"  POINT(): {point_format}")
print(f"  Degree-minute-second: {degree_format}")
print(f"  Comma-separated: {comma_format}")
print(f"  Semicolon-separated: {semicolon_format}")


## 3. Cleaning Pipeline

Now we'll implement a step-by-step cleaning pipeline:


In [ ]:
# Create a copy for cleaning
df_clean = df_raw.copy()
initial_rows = len(df_clean)
print(f"Starting with {initial_rows} rows")


### Step 1: Remove test/invalid rows


In [ ]:
# Identify invalid rows (containing test data patterns)
invalid_patterns = [
    r'^\?\?\?',  # Starts with ???
    r'not a date',  # Invalid date text
    r'^\s*$'  # Completely empty rows
]

# Check for rows with invalid patterns in key columns
invalid_mask = pd.Series([False] * len(df_clean))

for col in ['facility_id', 'facility_name', 'licence_issue_date', 'inspection_date']:
    for pattern in invalid_patterns:
        invalid_mask |= df_clean[col].str.contains(pattern, na=False, regex=True)

# Also check for rows where all important columns are empty/null
key_cols = ['facility_id', 'facility_name', 'region']
invalid_mask |= df_clean[key_cols].isnull().all(axis=1) | (df_clean[key_cols] == '').all(axis=1)

invalid_count = invalid_mask.sum()
print(f"Removing {invalid_count} invalid/test rows")
df_clean = df_clean[~invalid_mask].copy()
print(f"Rows remaining: {len(df_clean)}")


### Step 2: Standardize facility_id


In [ ]:
def clean_facility_id(facility_id):
    """Clean facility ID: extract only numbers and convert to int"""
    if pd.isna(facility_id) or facility_id == '' or str(facility_id).strip() == 'NULL':
        return None
    
    id_str = str(facility_id).strip()
    
    # Remove leading #
    if id_str.startswith('#'):
        id_str = id_str[1:]
    
    # Extract only digits
    digits = re.sub(r'[^0-9]', '', id_str)
    
    if digits == '':
        return None
    
    # Convert to int
    try:
        return int(digits)
    except ValueError:
        return None

df_clean['facility_id_clean'] = df_clean['facility_id'].apply(clean_facility_id)

# For deduplication, create a normalized ID (for comparison only)
def normalize_id_for_dedup(fid):
    """Create normalized ID for duplicate detection"""
    if pd.isna(fid):
        return None
    # Since IDs are now int, just convert to string for comparison
    return str(int(fid)) if pd.notna(fid) else None

df_clean['facility_id_normalized'] = df_clean['facility_id_clean'].apply(normalize_id_for_dedup)

print(f"Facility IDs cleaned (extracted numbers, converted to int). Sample:")
print(df_clean[['facility_id', 'facility_id_clean']].head(20))


In [ ]:
def reverse_region_name(region_text):
    """Reverse a region name that's been written backwards"""
    # Mapping of reversed region names to correct ones
    reversed_regions = {
        'semaj': 'St. James',
        'nhoj': 'St. John',
        'retep': 'St. Peter',
        'werdna': 'St. Andrew',
        'leahcim': 'St. Michael',
        'hpesoj': 'St. Joseph',
        'egroeg': 'St. George',
        'hcruhc tsirhc': 'Christ Church',
        'tsirhc hcruhc': 'Christ Church',
    }
    
    region_lower = region_text.lower().strip()
    
    # Try exact match first
    if region_lower in reversed_regions:
        return reversed_regions[region_lower]
    
    # Try partial match (e.g., 'semaj' in '(semaj)')
    for reversed_key, correct_name in reversed_regions.items():
        if reversed_key in region_lower:
            return correct_name
    
    return None

def clean_facility_name(name):
    """Clean facility name: fix abbreviations, reverse prefixes, move suffixes, remove special chars"""
    if pd.isna(name) or name == '':
        return None
    
    name_str = str(name)
    
    # Remove newlines and normalize whitespace
    cleaned = re.sub(r'\s+', ' ', name_str)
    cleaned = cleaned.replace('\n', ' ').replace('\r', ' ')
    
    # Handle backwards prefixes in parentheses: (Semaj) -> St. James, (nhoj) -> St. John
    # Find patterns like (semaj), (nhoj), etc.
    paren_pattern = r'\(([^)]+)\)'
    matches = list(re.finditer(paren_pattern, cleaned, re.IGNORECASE))
    
    # Process matches in reverse order to maintain indices
    replacements = []
    for match in reversed(matches):
        content = match.group(1).strip()
        # Check if it's a backwards region name
        reversed_region = reverse_region_name(content)
        if reversed_region:
            replacements.append((match.start(), match.end(), reversed_region))
        # Check if it's a suffix like (St.), (st.), (ST.)
        elif re.match(r'^[Ss]t\.?\s*$', content):
            # Move suffix to front
            suffix = content.strip()
            cleaned = cleaned[:match.start()] + cleaned[match.end():]
            cleaned = f"{suffix} {cleaned}".strip()
    
    # Apply region replacements
    for start, end, replacement in replacements:
        cleaned = cleaned[:start] + replacement + cleaned[end:]
    
    # Fix common abbreviations: Hosp. -> Hospital, Clin. -> Clinic
    cleaned = re.sub(r'\bHosp\.\b', 'Hospital', cleaned)
    cleaned = re.sub(r'\bClin\.\b', 'Clinic', cleaned)
    
    # Remove emojis and special unicode characters first
    cleaned = ''.join(char for char in cleaned if ord(char) < 128)
    
    # Remove all special characters (keep only alphanumeric, spaces, commas, periods, hyphens, apostrophes)
    cleaned = re.sub(r'[^a-zA-Z0-9\s,\.\-\']', '', cleaned)
    
    # Normalize whitespace again
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    return cleaned if cleaned else None

df_clean['facility_name_clean'] = df_clean['facility_name'].apply(clean_facility_name)

print("Sample of cleaned facility names:")
comparison = pd.DataFrame({
    'Original': df_clean['facility_name'].head(20),
    'Cleaned': df_clean['facility_name_clean'].head(20)
})
print(comparison[comparison['Original'] != comparison['Cleaned']].head(10))


### Step 4: Standardize facility_type


In [ ]:
# Mapping of variants to standard types
facility_type_mapping = {
    'hosp.': 'Hospital',
    'hospital': 'Hospital',
    'health ctr.': 'Health Centre',
    'health centre': 'Health Centre',
    'health center': 'Health Centre',
    'community health ctr.': 'Community Health Centre',
    'community health centre': 'Community Health Centre',
    'community health center': 'Community Health Centre',
    'chc': 'Community Health Centre',
    'polyclinic': 'Polyclinic',
    'clinic': 'Clinic',
    'health centre': 'Health Centre',
    'infirmary': 'Infirmary'
}

def standardize_facility_type(fac_type):
    """Standardize facility type"""
    if pd.isna(fac_type) or fac_type == '':
        return None
    
    fac_type_str = str(fac_type).strip()
    
    # Try to match (case-insensitive)
    fac_type_lower = fac_type_str.lower()
    
    # Check for exact matches first
    for key, value in facility_type_mapping.items():
        if key in fac_type_lower:
            return value
    
    # If no match, try to clean and return
    # Remove emojis and special chars
    cleaned = ''.join(char for char in fac_type_str if ord(char) < 128)
    return cleaned.strip() if cleaned.strip() else None

df_clean['facility_type_clean'] = df_clean['facility_type'].apply(standardize_facility_type)

print("Facility type standardization:")
print(f"\nBefore: {df_clean['facility_type'].value_counts().head(15)}")
print(f"\nAfter: {df_clean['facility_type_clean'].value_counts()}")


### Step 5: Parse capacity


In [ ]:
def parse_capacity(capacity):
    """Parse capacity to numeric value"""
    if pd.isna(capacity) or capacity == '':
        return None, None
    
    cap_str = str(capacity).strip()
    
    # Handle N/A and unknown
    if cap_str.upper() in ['N/A', 'NAN', 'UNKNOWN', 'NONE']:
        return None, None
    
    # Handle "ten beds" pattern
    if 'ten' in cap_str.lower():
        return 10, 'beds'
    
    # Extract number
    number_match = re.search(r'(\d+)', cap_str)
    if number_match:
        number = int(number_match.group(1))
        
        # Determine unit
        cap_lower = cap_str.lower()
        if 'bed' in cap_lower:
            unit = 'beds'
        elif 'cot' in cap_lower:
            unit = 'cots'
        elif 'patient' in cap_lower:
            unit = 'patients'
        elif 'capacity' in cap_lower:
            unit = 'capacity'
        else:
            unit = 'units'  # default
        
        return number, unit
    
    return None, None

capacity_results = df_clean['capacity'].apply(parse_capacity)
df_clean['capacity_numeric'] = [r[0] for r in capacity_results]
df_clean['capacity_unit'] = [r[1] for r in capacity_results]

print("Capacity parsing summary:")
print(f"Successfully parsed: {df_clean['capacity_numeric'].notna().sum()}")
print(f"\nSample parsed capacities:")
print(df_clean[['capacity', 'capacity_numeric', 'capacity_unit']].head(30))


In [ ]:
def reverse_text(text):
    """Reverse a text string"""
    return text[::-1]

def normalize_region(region):
    """Normalize region name, handling backwards text"""
    if pd.isna(region) or region == '':
        return None
    
    region_str = str(region).strip()
    region_lower = region_str.lower()
    
    # Mapping of known regions (Barbados parishes) including reversed variants
    canonical_regions = {
        'st. james': 'St. James', 'st james': 'St. James', 'st.james': 'St. James',
        'st. james parish': 'St. James',
        'semaj .ts': 'St. James', 'semaj .tS': 'St. James', 'semaj': 'St. James',
        'retep .ts': 'St. Peter', 'retep': 'St. Peter',
        'st. peter': 'St. Peter', 'st peter': 'St. Peter', 'st.peter': 'St. Peter',
        'st. peter parish': 'St. Peter',
        'werdna .ts': 'St. Andrew', 'werdna .tS': 'St. Andrew', 'werdna': 'St. Andrew',
        'st. andrew': 'St. Andrew', 'st andrew': 'St. Andrew', 'st.andrew': 'St. Andrew',
        'st. andrew parish': 'St. Andrew',
        'leahcim .ts': 'St. Michael', 'leahcim': 'St. Michael',
        'st. michael': 'St. Michael', 'st michael': 'St. Michael', 'st.michael': 'St. Michael',
        'st. michael parish': 'St. Michael',
        'hpesoj .ts': 'St. Joseph', 'hpesoj': 'St. Joseph', 'nhoj .ts': 'St. John', 'nhoj': 'St. John',
        'st. joseph': 'St. Joseph', 'st joseph': 'St. Joseph', 'st.joseph': 'St. Joseph',
        'st. joseph parish': 'St. Joseph',
        'st. john': 'St. John', 'st john': 'St. John', 'st.john': 'St. John',
        'st. john parish': 'St. John',
        'egroeg .ts': 'St. George', 'egroeg': 'St. George',
        'st. george': 'St. George', 'st george': 'St. George', 'st.george': 'St. George',
        'st. george parish': 'St. George',
        'hcruhc tsirhc': 'Christ Church', 'hcruhC tsirhC': 'Christ Church',
        'christ church': 'Christ Church', 'christchurch': 'Christ Church',
        'christ church parish': 'Christ Church',
        'st. lucy': 'St. Lucy', 'st lucy': 'St. Lucy', 'st.lucy': 'St. Lucy',
        'st. lucy parish': 'St. Lucy',
    }
    
    # Check direct mapping
    if region_lower in canonical_regions:
        return canonical_regions[region_lower]
    
    # Try case-insensitive matching
    for key, value in canonical_regions.items():
        if key in region_lower:
            return value
    
    # Try reversing the text and checking
    reversed_text = reverse_text(region_str)
    reversed_lower = reversed_text.lower().strip()
    
    # Check if reversed text matches a known region
    for key, value in canonical_regions.items():
        if key in reversed_lower or reversed_lower in key:
            return value
    
    # If still no match, try to standardize case
    words = region_str.split()
    normalized = ' '.join(word.capitalize() if word.lower() not in ['st', 'st.', 'and'] else word.title() 
                         for word in words)
    
    return normalized

df_clean['region_clean'] = df_clean['region'].apply(normalize_region)

print("Region normalization summary:")
print(f"\nBefore (unique): {df_clean['region'].nunique()}")
print(f"\nAfter (unique): {df_clean['region_clean'].nunique()}")
print(f"\nCleaned regions:\n{df_clean['region_clean'].value_counts()}")


In [ ]:
def parse_date(date_str):
    """Parse various date formats to datetime"""
    if pd.isna(date_str) or date_str == '':
        return None
    
    date_str = str(date_str).strip()
    
    # List of date formats to try
    date_formats = [
        '%Y-%m-%d',           # 2016-02-18
        '%d-%m-%y',           # 04-01-21
        '%d-%m-%Y',           # 04-01-2021
        '%d/%m/%y',           # 18/05/18
        '%d/%m/%Y',           # 18/05/2018
        '%Y%m%d',             # 20160217
        '%d %b %Y',           # 08 Mar 2024
        '%d %B %Y',           # March 08 2024
        '%B %d %Y',           # November 21 2020
        '%b %d %Y',           # Mar 28 2024
        '%d %B %y',           # March 08 24
        '%Y%m%d',             # 20230822
        '%y%m%d',             # 210617
    ]
    
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt, errors='raise')
        except (ValueError, TypeError):
            continue
    
    # Try pandas flexible parsing as last resort
    try:
        return pd.to_datetime(date_str, errors='raise', infer_datetime_format=True)
    except:
        return None

df_clean['licence_issue_date_clean'] = df_clean['licence_issue_date'].apply(parse_date)
df_clean['inspection_date_clean'] = df_clean['inspection_date'].apply(parse_date)

print("Date parsing summary:")
print(f"Licence dates parsed: {df_clean['licence_issue_date_clean'].notna().sum()} / {len(df_clean)}")
print(f"Inspection dates parsed: {df_clean['inspection_date_clean'].notna().sum()} / {len(df_clean)}")

# Flag date anomalies
df_clean['date_anomaly'] = False
df_clean.loc[
    (df_clean['inspection_date_clean'].notna()) & 
    (df_clean['licence_issue_date_clean'].notna()) &
    (df_clean['inspection_date_clean'] < df_clean['licence_issue_date_clean']),
    'date_anomaly'
] = True

print(f"\nDate anomalies (inspection before licence): {df_clean['date_anomaly'].sum()}")

# Check for future dates (assuming current date is 2024-12-31)
current_date = pd.Timestamp('2024-12-31')
future_licences = (df_clean['licence_issue_date_clean'] > current_date).sum()
future_inspections = (df_clean['inspection_date_clean'] > current_date).sum()
print(f"Future licence dates: {future_licences}")
print(f"Future inspection dates: {future_inspections}")


In [ ]:
def parse_gps_coordinates(gps_str):
    """Parse various GPS formats to decimal degrees (lat, lon)"""
    if pd.isna(gps_str) or gps_str == '' or str(gps_str).upper() == 'N/A':
        return None, None
    
    gps_str = str(gps_str).strip()
    
    # Handle POINT format: POINT(-58.84001 12.87196)
    point_match = re.search(r'POINT\(([-\d.]+)\s+([-\d.]+)\)', gps_str, re.IGNORECASE)
    if point_match:
        lon = float(point_match.group(1))
        lat = float(point_match.group(2))
        return lat, lon
    
    # Handle comma-separated: "13.08576, -58.75331" or "-58.82307, 13.00952"
    comma_match = re.search(r'([-\d.]+)\s*,\s*([-\d.]+)', gps_str)
    if comma_match:
        val1 = float(comma_match.group(1))
        val2 = float(comma_match.group(2))
        # Determine which is lat and which is lon based on ranges
        # Latitude: -90 to 90, Longitude: -180 to 180
        # For Barbados: lat ~12-13, lon ~-59 to -58
        if -90 <= val1 <= 90 and -180 <= val2 <= 180:
            # First is likely lat, second is lon
            if abs(val1) > abs(val2) and val1 > 0:  # If first is clearly lat range
                return val1, val2
            elif abs(val2) < abs(val1):  # Second is likely lat (smaller absolute value for Barbados)
                return val2, val1
            else:
                return val1, val2
        else:
            # Assume first is lat, second is lon
            return val1, val2
    
    # Handle semicolon-separated: -58.71602;12.98428
    semicolon_match = re.search(r'([-\d.]+)\s*;\s*([-\d.]+)', gps_str)
    if semicolon_match:
        val1 = float(semicolon_match.group(1))
        val2 = float(semicolon_match.group(2))
        # Similar logic to comma
        if abs(val1) < abs(val2) and val1 > 0:  # First is likely lat
            return val1, val2
        else:
            return val2, val1
    
    # Handle degree-minute-second: 13°12′43″N 58°51′50″W
    dms_match = re.search(r'(\d+)°(\d+)′(\d+)″([NS])\s+(\d+)°(\d+)′(\d+)″([EW])', gps_str)
    if dms_match:
        lat_deg = int(dms_match.group(1))
        lat_min = int(dms_match.group(2))
        lat_sec = int(dms_match.group(3))
        lat_dir = dms_match.group(4)
        
        lon_deg = int(dms_match.group(5))
        lon_min = int(dms_match.group(6))
        lon_sec = int(dms_match.group(7))
        lon_dir = dms_match.group(8)
        
        lat = lat_deg + lat_min/60 + lat_sec/3600
        if lat_dir == 'S':
            lat = -lat
        
        lon = lon_deg + lon_min/60 + lon_sec/3600
        if lon_dir == 'W':
            lon = -lon
        
        return lat, lon
    
    return None, None

gps_results = df_clean['gps_location'].apply(parse_gps_coordinates)
df_clean['latitude'] = [r[0] for r in gps_results]
df_clean['longitude'] = [r[1] for r in gps_results]

print("GPS parsing summary:")
print(f"Successfully parsed: {df_clean['latitude'].notna().sum()} / {len(df_clean)}")

# Validate coordinate ranges (Barbados approximate bounds)
valid_coords = (
    (df_clean['latitude'] >= 12.5) & (df_clean['latitude'] <= 13.5) &
    (df_clean['longitude'] >= -59.7) & (df_clean['longitude'] <= -58.5)
) | df_clean['latitude'].isna()

print(f"Coordinates within Barbados bounds: {valid_coords.sum()} / {len(df_clean)}")
print(f"Out-of-range coordinates: {(~valid_coords).sum()}")

# Set out-of-range to None
df_clean.loc[~valid_coords, 'latitude'] = None
df_clean.loc[~valid_coords, 'longitude'] = None

print(f"\nFinal valid coordinates: {df_clean['latitude'].notna().sum()}")


### Step 9: Handle duplicates


In [ ]:
# Remove exact duplicates first
before_exact_dup = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=df_clean.columns.tolist(), keep='first')
exact_dup_removed = before_exact_dup - len(df_clean)
print(f"Exact duplicates removed: {exact_dup_removed}")

# Handle near-duplicates (same facility name but different IDs)
# Create a function to identify potential duplicates based on name similarity
def create_name_key(name):
    """Create a normalized key for name comparison"""
    if pd.isna(name):
        return None
    # Remove special chars, lowercase, remove common suffixes
    name_key = str(name).lower()
    name_key = re.sub(r'[^\w\s]', '', name_key)
    name_key = re.sub(r'\s+', ' ', name_key).strip()
    # Remove common abbreviations
    name_key = name_key.replace('st.', 'st').replace('st ', 'st')
    name_key = name_key.replace('hosp.', 'hospital').replace('hosp ', 'hospital')
    name_key = name_key.replace('clin.', 'clinic').replace('clin ', 'clinic')
    return name_key

df_clean['name_key'] = df_clean['facility_name_clean'].apply(create_name_key)

# Find potential duplicates (same name key and region)
potential_dups = df_clean.groupby(['name_key', 'region_clean']).size()
potential_dups = potential_dups[potential_dups > 1].reset_index(name='dup_count')

print(f"\nPotential duplicate groups (same name + region): {len(potential_dups)}")

# For each duplicate group, keep the record with most complete data
def completeness_score(row):
    """Calculate completeness score (higher = more complete)"""
    score = 0
    if pd.notna(row.get('facility_name_clean')): score += 1
    if pd.notna(row.get('facility_type_clean')): score += 1
    if pd.notna(row.get('capacity_numeric')): score += 1
    if pd.notna(row.get('licence_issue_date_clean')): score += 1
    if pd.notna(row.get('inspection_date_clean')): score += 1
    if pd.notna(row.get('latitude')): score += 1
    if pd.notna(row.get('remarks')) and str(row.get('remarks')).strip() not in ['', '-', 'N/A']: score += 1
    return score

df_clean['completeness_score'] = df_clean.apply(completeness_score, axis=1)

# Remove duplicates keeping the most complete record
df_clean = df_clean.sort_values('completeness_score', ascending=False)
df_clean = df_clean.drop_duplicates(subset=['name_key', 'region_clean'], keep='first')

print(f"Rows after duplicate removal: {len(df_clean)}")
df_clean = df_clean.drop(columns=['name_key', 'completeness_score'])


### Step 10: Create final cleaned dataset


In [ ]:
# Clean remarks - remove strange characters
def clean_remarks(remark):
    """Clean remarks by removing special characters and emojis"""
    if pd.isna(remark) or remark == '':
        return None
    
    remark_str = str(remark).strip()
    
    # Remove emojis and special unicode characters
    cleaned = ''.join(char for char in remark_str if ord(char) < 128)
    
    # Remove characters except alphanumeric, spaces, hyphens, commas, periods
    cleaned = re.sub(r'[^a-zA-Z0-9\s\-,\.]', '', cleaned)
    
    # Normalize whitespace
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    # Handle common empty indicators
    if cleaned in ['', '-', 'N/A', 'NULL']:
        return None
    
    return cleaned if cleaned else None

df_clean['remarks_clean'] = df_clean['remarks'].apply(clean_remarks)

# Create final dataset with cleaned columns
df_final = pd.DataFrame({
    'facility_id': df_clean['facility_id_clean'],
    'facility_name': df_clean['facility_name_clean'],
    'facility_type': df_clean['facility_type_clean'],
    'capacity_numeric': df_clean['capacity_numeric'],
    'capacity_unit': df_clean['capacity_unit'],
    'region': df_clean['region_clean'],
    'licence_issue_date': df_clean['licence_issue_date_clean'],
    'inspection_date': df_clean['inspection_date_clean'],
    'latitude': df_clean['latitude'],
    'longitude': df_clean['longitude'],
    'remarks': df_clean['remarks_clean'],  # Use cleaned remarks
    'date_anomaly': df_clean['date_anomaly']  # Flag for date issues
})

# Replace empty strings with NaN
df_final = df_final.replace(['', 'NULL', 'N/A', '-'], np.nan)

print("Final dataset shape:", df_final.shape)
print("\nData types:")
print(df_final.dtypes)
print("\nMissing values:")
print(df_final.isnull().sum())


### Step 11: Final validation and summary


In [ ]:
print("=== CLEANING SUMMARY ===")
print(f"\nInitial rows: {initial_rows}")
print(f"Final rows: {len(df_final)}")
print(f"Rows removed: {initial_rows - len(df_final)}")
print(f"Retention rate: {len(df_final)/initial_rows*100:.1f}%")

print(f"\n=== DATA QUALITY METRICS ===")
print(f"\nFacility IDs: {df_final['facility_id'].notna().sum()} / {len(df_final)} ({df_final['facility_id'].notna().sum()/len(df_final)*100:.1f}%)")
print(f"Facility Names: {df_final['facility_name'].notna().sum()} / {len(df_final)} ({df_final['facility_name'].notna().sum()/len(df_final)*100:.1f}%)")
print(f"Facility Types: {df_final['facility_type'].notna().sum()} / {len(df_final)} ({df_final['facility_type'].notna().sum()/len(df_final)*100:.1f}%)")
print(f"Capacity: {df_final['capacity_numeric'].notna().sum()} / {len(df_final)} ({df_final['capacity_numeric'].notna().sum()/len(df_final)*100:.1f}%)")
print(f"Region: {df_final['region'].notna().sum()} / {len(df_final)} ({df_final['region'].notna().sum()/len(df_final)*100:.1f}%)")
print(f"Licence Date: {df_final['licence_issue_date'].notna().sum()} / {len(df_final)} ({df_final['licence_issue_date'].notna().sum()/len(df_final)*100:.1f}%)")
print(f"Inspection Date: {df_final['inspection_date'].notna().sum()} / {len(df_final)} ({df_final['inspection_date'].notna().sum()/len(df_final)*100:.1f}%)")
print(f"Coordinates: {df_final['latitude'].notna().sum()} / {len(df_final)} ({df_final['latitude'].notna().sum()/len(df_final)*100:.1f}%)")

print(f"\n=== DISTRIBUTIONS ===")
print(f"\nFacility Types:\n{df_final['facility_type'].value_counts()}")
print(f"\nRegions:\n{df_final['region'].value_counts()}")
print(f"\nCapacity Units:\n{df_final['capacity_unit'].value_counts()}")
print(f"\nDate Anomalies: {df_final['date_anomaly'].sum()}")

print(f"\n=== STATISTICS ===")
if df_final['capacity_numeric'].notna().sum() > 0:
    print(f"\nCapacity statistics:")
    print(df_final['capacity_numeric'].describe())


## 4. Save Cleaned Dataset


In [ ]:
# Save to CSV
df_final.to_csv('cleaned_health_registry.csv', index=False)
print("Cleaned dataset saved to 'cleaned_health_registry.csv'")
print(f"Final shape: {df_final.shape}")

# Display sample of cleaned data
print("\nSample of cleaned data:")
df_final.head(10)
